<a href="https://colab.research.google.com/github/csernaquech/CursoDMC/blob/main/ViolenciaCEM_RLN_Tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import math

# Configuración de tamaño de gráficos matplotlib
import matplotlib.pyplot as plt

# Cargar seaborn
import seaborn as sns

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# os proporciona funciones para interactuar con el sistema operativo y realizar operaciones relacionadas con archivos y directorios.
import os

dfv = pd.read_csv('denuncias_procesadas_colab_compleado.csv',delimiter=';',encoding='ISO-8859-1')



In [ ]:
dfv.describe()

In [ ]:
# Creando carpetas
# nombre_carpeta = 'linear_regression'
# os.makedirs(nombre_carpeta)
# os.makedirs(nombre_carpeta + '/data')
# os.makedirs(nombre_carpeta + '/plots')
# os.makedirs(nombre_carpeta + '/results')

In [ ]:
target = 'FEMINICIDIO'

In [ ]:
variables_numericas = dfv.select_dtypes(include=['number']).columns.tolist()
variables_numericas


In [ ]:
variables_numericas.remove('FEMENICIDIO')
variables_numericas

In [ ]:
# Identificar las variables categóricas utilizando el atributo dtypes
variables_categoricas = dfv.select_dtypes(include=['object']).columns.tolist()
variables_categoricas

In [ ]:
# Guardando variables en una tabla
dataResume = pd.DataFrame({'feature' : variables_numericas, 'type' : 'numeric'})
dataResume = dataResume.append(pd.DataFrame({'feature' : [target], 'type' : 'target'}))
dataResume.to_csv('datanumericaViolencia.csv', index = 0)



In [ ]:
pip install seaborn

In [ ]:
plt.figure(figsize=(4, 6))  # Tamaño del gráfico (ancho x alto en pulgadas)

# Gráfico de distribución
sns.displot(dfv['PONDER_FICHA_RIESGO'], height=6, aspect=3, kde=True)
plt.title("Distribución del Precio de los inmuebles", fontsize=12)  # Título del gráfico
plt.xlabel("Precio", fontsize=10)  # Nombre del eje x
plt.ylabel("Frecuencia", fontsize=10)  # Nombre del eje y
plt.xticks(range(0, 40, 1))

# #plt.savefig("dist_MEDV.png")  # Guarda el gráfico como un archivo PNG
# plt.show()




In [ ]:
plt.figure(figsize=(4, 6))  # Tamaño del gráfico (ancho x alto en pulgadas)

for col in variables_numericas:
  # Gráfico de distribución
  sns.displot(dfv[col], height=6, aspect=3, kde=True)
  plt.title("Distribución de " + col, fontsize=12)  # Título del gráfico
  plt.xlabel(col, fontsize=10)  # Nombre del eje x
  plt.ylabel("Frecuencia", fontsize=10)  # Nombre del eje y
  plt.savefig("linear_regression/plots/dist_" + col + ".png")  # Guarda el gráfico como un archivo PNG
  plt.close()

In [ ]:
#ENTRENANDO AL MODELO
# Muestreo
#La función train_test_split de scikit-learn se utiliza para dividir un conjunto de datos en subconjuntos de train y test.
from sklearn.model_selection import train_test_split

train, test = train_test_split(dfv, # Base de datos
                               train_size = 0.7, # Especificar el tamaño de train/test
                               random_state = 123) # Semilla aleatoria

print(f"Tamaño del total de datos: {len(dfv)}")
print(f"Tamaño de train: {len(train)}")
print(f"Tamaño de test: {len(test)}")

In [ ]:
# Crear scatterplot
plt.figure(figsize=(6, 4))
sns.regplot(x = train.FEMENICIDIO, y = train.VICT_EDAD, scatter_kws={"alpha": 0.5})

# Personalizar el título y los nombres de los ejes
plt.title(f"Scatterplot NOX vs Precio del inmueble")
plt.xlabel("Variable NOX")
plt.ylabel("Precio del inmueble")

# Mostrar el gráfico
plt.show()

In [ ]:
pdResume = train.describe().transpose()
pdResume.reset_index(inplace = True)
pdResume.rename(columns = {'index' : 'feature',
                           '25%' : 'Q1',
                           '50%' : 'median',
                           '75%' : 'Q3'}, inplace = True)

#pdResume

pdResume[['missing_rate']] = 1 - pdResume[['count']]/train.shape[0]
pdResume.sort_values(by = 'missing_rate', ascending = False, inplace = True)

In [ ]:
correlations = train.corr()[['FEMENICIDIO']]
correlations.reset_index(inplace = True)
correlations.rename(columns = {'index' : 'feature',
                               'FEMENICIDIO' : 'corr'}, inplace = True)




In [ ]:
# Cruzando correlaciones con la tabla de resumen original
univariate = pd.merge(pdResume, correlations, on = 'feature')
univariate.head()

In [ ]:
# Creamos la columna correlacion absoluta
univariate['abs_corr'] = univariate['corr'].apply(lambda x: abs(x))
univariate.sort_values(by = 'abs_corr', ascending = False, inplace=True)
univariate.head()

In [ ]:
# Guardando resultados
univariate.to_csv('linear_regression/results/univariate_assesment.csv', index = 0)

In [ ]:
preselection = univariate.loc[(univariate.abs_corr >= 0.2) & (univariate.missing_rate <= 0.8)]
print(f"{len(preselection)} variables de {len(univariate)} pasan el proceso de pre-selección")


preselection

In [ ]:
preselection = univariate.loc[(univariate.abs_corr >= 0.2) & (univariate.missing_rate <= 0.8)]
print(f"{len(preselection)} variables de {len(univariate)} pasan el proceso de pre-selección")

In [ ]:
#Leyendo tabla resumen
univariate_II = pd.read_csv('linear_regression/results/univariate_assesment.csv')
univariate_II.head()

In [ ]:
##4.1. Tratamiento de outliers


univariate_II['ric']     = univariate_II['Q3'] - univariate_II['Q1']
univariate_II['min_ric'] = univariate_II['Q1'] - 1.5*univariate_II['ric']
univariate_II['max_ric'] = univariate_II['Q3'] + 1.5*univariate_II['ric']

univariate_II.head()

In [ ]:
# Tratamiento de outliers por RIC

for col in preselection.feature.tolist():
  desc = univariate_II.loc[univariate_II.feature == col]

  lower_limit = desc.min_ric.values[0]
  upper_limit = desc.max_ric.values[0]

  train[col + '_tric'] = train[col].apply(lambda x: lower_limit if x <= lower_limit else
                                                    upper_limit if x >= upper_limit else
                                                    x)
  test[col + '_tric']  = test[col].apply(lambda x: lower_limit if x <= lower_limit else
                                                    upper_limit if x >= upper_limit else
                                                    x)
train.head()

In [ ]:
#Revisemos los resultados

# Crear figura y subplots
fig, axes = plt.subplots(2, 1, figsize=(6, 4))

ax1 = axes[0]
sns.boxplot(data=train[['FEMENICIDIO']], orient="h", ax=ax1)
ax1.set_title('Variable Original')

ax2 = axes[1]
sns.boxplot(data=train[['FEMENICIDIO_tric']], orient="h", ax=ax2)
ax2.set_title('Variable Tratada')
# Ajustar espaciado entre subplots
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [ ]:
#4.2. Tratamiento de Missings
# Variables hasta el momento
actual_features = [x + '_tric' for x in preselection.feature.tolist()]

# Tratamiento en train
train[[x + '_imp' for x in actual_features]] = train[actual_features].fillna(train[actual_features].median())

# Tratamiento en test
test[[x + '_imp' for x in actual_features]] = test[actual_features].fillna(train[actual_features].median())


In [ ]:
train[[x + '_imp' for x in actual_features]].head(30)


In [ ]:
#4.3. Reescalamiento de datos

# Variables hasta el momento
actual_features = [x + '_imp' for x in actual_features]




In [ ]:
actual_features

# estandarizando las variables para mantener todas en una misma escala
train[[x + '_std' for x in actual_features]] = (train[actual_features] - train[actual_features].mean())/train[actual_features].std()
test[[x + '_std' for x in actual_features]] = (test[actual_features] - train[actual_features].mean())/train[actual_features].std()

In [ ]:
#5. Modelo de regresión Lineal
#Regresión lineal por MCO
[x for x in train.columns.tolist() if 'FEMENICIDIO' in x]



In [ ]:
# Variables finalmente tratadas
actual_features = [x + '_std' for x in actual_features]
#actual_features.remove('FEMENICIDIO' + 'tric_imp_std')

In [ ]:
actual_features